In [1]:
import pandas as pd
import numpy as np

# format sig figs
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# Append Observed to Modeled (previously completed)
## Add the script here...

In [2]:
#############################
# Load the observed records
# 
obs = pd.read_csv(r'R:\FastTrips\CHTS and OBS ft_output\OBS_ft_output_w_stops.csv')
obs.fillna('',inplace=True)
obs['person_id'] = obs['person_id'].apply(lambda x: x.split("_")[-1]).astype('str')
obs['A_id_num'] = obs['A_id_num'].replace("",0)
obs['A_id_num'] = obs['A_id_num'].astype('int')
obs = obs.drop('trip_list_id_num',axis=1)

# ADD THE PROPER trip_list_id_num
obsinput = pd.read_csv(r'J:\Projects\FasTrips\obs\input\1.0\OBS_fasttrips_demand_v1.0\trip_list.txt')
obsinput['person_id'] = obsinput['person_id'].astype('int').astype('str')

obs = pd.merge(obs, obsinput[['person_id','trip_list_id_num']], on='person_id')

# Make sure observed matches modeled


##########################
# On Board Survey Results
model = pd.read_csv(r'J:\Projects\FasTrips\obs\output\OBS_fasttrips_demand_v1.0_stochastic_iter2_cap\chosenpaths_links.csv')
model['route_id'].fillna("",inplace=True)
# FOR NOW, assume we look at only the last iteration of path sets ?
model = model[model['iteration'] == model['iteration'].max()]
model['person_id'] = model['person_id'].astype('int').astype('str')


#########################
# Pathsets
pathset_links = pd.read_csv(r'J:\Projects\FasTrips\obs\output\OBS_fasttrips_demand_v1.0_stochastic_iter2_cap\pathset_links.csv')
pathset_links['route_id'].fillna("",inplace=True)
pathset_links = pathset_links[pathset_links['iteration'] == pathset_links['iteration'].max()]
pathset_links['person_id'] = pathset_links['person_id'].astype('int').astype('str')
pathset_links['unique_id'] = pathset_links['person_id']

# For OBS
pathset_paths = pd.read_csv(r'J:\Projects\FasTrips\obs\output\OBS_fasttrips_demand_v1.0_stochastic_iter2_cap\pathset_paths.csv')
pathset_paths['person_id'] = pathset_paths['person_id'].astype('int').astype('str')
pathset_paths['unique_id'] = pathset_paths['person_id'].astype('str') +"_"+ pathset_paths['trip_list_id_num'].astype('str')

In [80]:
# Load data CHTS

# # For testing, use a known list of trips, e.g., fast trips output
# obs = pd.read_csv(r'J:\Projects\FasTrips\chts\output\CHTS_fasttrips_demand_v0.1_stochastic_iter2_nocap\chosenpaths_links.csv')
# # Won't need to do this with real observed data
# obs['route_id'].fillna("",inplace=True)
# obs = obs[obs['iteration'] == obs['iteration'].max()]

# CHTS
# Load the model records
# model = pd.read_csv(r'J:\Projects\FasTrips\chts\output\CHTS_fasttrips_demand_v0.1_stochastic_iter2_nocap\chosenpaths_links.csv')
# model['route_id'].fillna("",inplace=True)
# # FOR NOW, assume we look at only the last iteration of path sets ?
# model = model[model['iteration'] == model['iteration'].max()]

# Load pathset links
## CHTS
# pathset_links = pd.read_csv(r'J:\Projects\FasTrips\tableau\path-set\pathset_links.csv')
# pathset_links['route_id'].fillna("",inplace=True)
# pathset_links = pathset_links[pathset_links['iteration'] == pathset_links['iteration'].max()]

# Create Unique ID

In [13]:
obs['unique_id'] = obs['person_id'].astype('str') +"_"+obs['trip_list_id_num'].astype('str')
model['unique_id'] = model['person_id'].astype('str') +"_"+model['trip_list_id_num'].astype('str')
pathset_links['unique_id'] = pathset_links['person_id'].astype('int').astype('str') +"_"+pathset_links['trip_list_id_num'].astype('str')
# pathset_paths['unique_id'] = pathset_paths['person_id'].astype('str') +"_"+pathset_paths['trip_list_id_num'].astype('str')

# Trim Data

In [14]:
pathset_links = pathset_links[pathset_links['pf_iteration'] == pathset_links['pf_iteration'].max()]

In [15]:
pathset_paths = pathset_paths[pathset_paths['pf_iteration'] == pathset_paths['pf_iteration'].max()]

In [16]:
# Make sure we only compare observed data that matched the output, in terms of trips
# Some trips were trimmed
obs = obs[obs['unique_id'].isin(model['unique_id'].values)]

In [24]:
# Since we modeled only a subset of trips, we can trim the pathset so that it includes
# only the modeled trips
# len(obs['unique_id'].isin(pathset_paths['unique_id'].values))



70988

In [26]:
# len(obs)

# Join correct transit route IDs to OBS data

# Add a transit agency lookup too

C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [96]:
def produce_path_fields(df, group):
    '''
    '''
    # create "path_routes"
    df['path_routes'] = df['route_id'].apply(lambda x: x.strip())
    path_routes = pd.DataFrame(df.groupby(group)['path_routes'].apply(lambda x: "%s" % ' '.join(x).strip()))
    
    result_df = pd.DataFrame(index=path_routes.index)
    result_df['path_routes'] = path_routes
    
    # create "path_modes"
    df['path_modes'] = df['mode'].apply(lambda x: x.strip())
    result_df['path_modes'] = pd.DataFrame(df.groupby(group)['mode'].apply(lambda x: "%s" % ' '.join(x).strip()))

    # Create "path_components"
    df['path_components'] = df['A_id'].astype('str')+" "+df['mode']+" "+df['route_id'] +"_"+ df['B_id'].astype('str')
    df['path_components'] = df['path_components'].apply(lambda x: x.strip())
    result_df['path_components'] = pd.DataFrame(df.groupby(group)['path_components'].apply(lambda x: "%s" % ' '.join(x).strip()))
    
    # Return ID field from index
    result_df['unique_id'] = result_df.index.get_level_values(0).values
    

    return result_df

# Produce joined fields from pathset link files 

In [97]:
# Create caluclated joined fields for the observed, modeled, and pathset links file

# Concatenate modes, route IDs, etc to produce unique trip identities
# Do this for each set of trips in the oberved data, as well as for the modeled, since we don't have the field produced by FT
observed_path = produce_path_fields(obs, group=['unique_id'])

In [29]:
modeled_path = produce_path_fields(model, group=['unique_id'])

### Weirdness with unique_id mismatch between model/obs and the pathset_links file

In [64]:
# also process the detailed pathset_links files, so each path in the pathset has a unique trip identity
new_pathset = produce_path_fields(pathset_links, group=['unique_id','pathnum'])

################################
# For some reason there is not a smooth overlap between the modeled pathset_link unique_ids and the model/observed data
# there are unique IDs missing from each, so I guess we just take an intersection of what's available for all of them?
print len(new_pathset)
print len(model)
print len(obs)

170530
111070
70988


In [65]:
obs = obs[obs['unique_id'].isin(new_pathset['unique_id'].values)]
model = model[model['unique_id'].isin(new_pathset['unique_id'].values)]
new_pathset = new_pathset[new_pathset['unique_id'].isin(obs['unique_id'].values)]
new_pathset = new_pathset[new_pathset['unique_id'].isin(model['unique_id'].values)]

In [66]:
print len(new_pathset)
print len(model)
print len(obs)

85723
42995
18669


## Compare if modeled/observed trips match, completed or partially

In [9]:
# Join the observed and modeled fields
df = pd.merge(observed_path, modeled_path, on='unique_id',suffixes=("_observed","_model"))

In [10]:
# Find rows with matching path routes
complete_route_match = df[df['path_routes_observed'] == df['path_routes_model']]
complete_mode_match = df[df['path_modes_observed'] == df['path_modes_model']]
# Add complete_agency_match when available
# complete_route_match = df[df['path_agency_observed'] == df['path_agency_model']]

In [11]:
## Extract order of transit routes taken
df['model_path_route_list'] = df['path_routes_model'].apply(lambda x: x.split(" "))
df['obs_path_route_list'] = df['path_routes_observed'].apply(lambda x: x.split(" "))

df['model_path_mode_list'] = df['path_modes_model'].apply(lambda x: x.split(" "))
df['obs_path_mode_list'] = df['path_modes_observed'].apply(lambda x: x.split(" "))

In [12]:
# Isolate transit modes only, because almost all trips have walk & transfer components
non_transit_modes = ['transfer','walk_access','walk_egress','bike_access','bike_egress',
                     'PNR_access','PNR_egress','KNR_access','KNR_egress']
df['model_transit_modes'] = df['model_path_mode_list'].apply(
    lambda row: [element for element in row if element not in non_transit_modes])
df['obs_transit_modes'] = df['obs_path_mode_list'].apply(
    lambda row: [element for element in row if element not in non_transit_modes])

In [13]:
# df['routes_in_path'].apply(lambda x: len(x.split(" ")))

# Get a 

# Routes
df.apply(lambda row: all(i in row['model_path_route_list'] for i in row['obs_path_route_list']), axis=1)
df['routes_intersection'] = [list(set(a).intersection(set(b))) for a, b in zip(df['model_path_route_list'], df['obs_path_route_list'])]

# All Modes
df.apply(lambda row: all(i in row['model_path_mode_list'] for i in row['obs_path_mode_list']), axis=1)
df['all_modes_intersection'] = [list(set(a).intersection(set(b))) for a, b in zip(df['model_path_mode_list'], df['obs_path_mode_list'])]

# Transit modes only
df.apply(lambda row: all(i in row['model_path_mode_list'] for i in row['obs_path_mode_list']), axis=1)
df['transit_modes_intersection'] = [list(set(a).intersection(set(b))) for a, b in zip(df['model_path_mode_list'], df['obs_path_mode_list'])]


# Agency Intersection

# All path components


# Exact Match of Path Routes, Modes, Components

In [14]:
# Exact match of path modes
complete_mode_match['complete_mode_match'] = 1
df = pd.merge(df, complete_mode_match[['unique_id','complete_mode_match']], how='left', on='unique_id')
df['complete_mode_match'].fillna(0,inplace=True)

C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [15]:
complete_route_match['complete_route_match'] = 1

In [16]:
df = pd.merge(df, complete_route_match[['unique_id','complete_route_match']], how='left', on='unique_id')

In [17]:
df['complete_route_match'] = df['complete_route_match'].replace('nan',0)


In [18]:
df['complete_route_match'].mean()

0.0

In [19]:
df['complete_mode_match'].mean()

0.2227124813854724

## Export fields to CSV for Tableau:
- 'complete_route_match' 0/1
- 'complete_mode_match' 0/1

# % trips with matching or partial matching routes

In [21]:
# Now we find the percent of trips with matching routes or partial matching routes
# df.mean()[['complete_match','partial_match']]
# Note, shuold be 100% when using the same data set, what's up??

#############
# Join the filtered data to the original results
df['common_mode_count'] = [len(row) for row in df['all_modes_intersection']]
df['common_transit_mode_count'] = [len(row) for row in df['transit_modes_intersection']]

# How many rows have at least one mode in common?
df['partial_mode_match'] = [1 if row > 0 else 0 for row in df['common_mode_count']]
df['partial_transit_mode_match'] = [1 if row > 0 else 0 for row in df['common_transit_mode_count']]

In [22]:

df['partial_mode_match'].mean()

0.9986763002592245

In [23]:
df['partial_transit_mode_match'].mean()

0.9986763002592245

## Export fields to CSV for Tableau:
- 'partial_mode_match' 0/1
- 'complete_mode_match' 0/1

# Compare probability of observed path to pathset


## Check if path exists in the pathset

In [67]:
def check_path_existence(row, pathset_df, unique_id, lookup_field):
    """
    Compare field contents from chosen paths vs entire path set,
    Return 1 if chosen path exists in pathset, 0 otherwise
    """
    
    if len(pathset_df[(pathset_df[unique_id] == row[unique_id]) & \
                        (pathset_df[lookup_field] == row[lookup_field])]) > 0:
        return 1
    else:
        return 0

In [72]:
route_match = observed_path.apply(check_path_existence, pathset_df=new_pathset, unique_id='unique_id', lookup_field='path_routes', axis=1)

In [73]:
mode_match = observed_path.apply(check_path_existence, pathset_df=new_pathset, unique_id='unique_id', lookup_field='path_modes', axis=1)

## To add: stop based data, origin, destination, and transfers, unique set of those?

In [82]:
# Mode match is the only one that should match right now because the transit route IDs aren't attached
mode_match.max()

1L

In [83]:
mode_match.mean()

0.018586950526722188

In [88]:
observed_path[observed_path['unique_id'] == '10036_3621']

,path_routes,path_modes,path_components,unique_id
unique_id,,,,
10036_3621,BART_1 28,walk_access heavy_rail walk_egress,2059.0 walk_access _16520.0 16520.0 heavy_rail...,10036_3621


In [87]:
new_pathset[new_pathset['unique_id'] == '10036_3621']

path_routes  \
unique_id  pathnum                                                      
10036_3621 0              38_74AC  38_70AC  56_JRNB  37_LAWB  30_292S   
           1             38_74AC  38_70AC  56_JPXNB  37_LAWB  30_292S   
           2        38_74AC  38_70AC  56_JPXNB  38_71SBAC  37_LAWB...   
           3        38_74AC  38_70AC  38_70AC  56_JPXNB  37_LAWB  ...   
           4        38_74AC  38_74AC  38_70AC  56_JPXNB  37_LAWB  ...   
           5        38_74AC  38_74AC  38_70AC  56_JPXNB  37_LAWB  ...   
           6        38_74AC  38_74AC  38_70AC  56_JPXNB  37_LAWB  ...   
           7        38_74AC  38_70AC  38_70AC  56_JPXNB  37_LAWB  ...   
           8        38_74AC  38_74AC  38_70AC  56_JPXNB  37_LAWB  ...   
           9        38_74AC  38_70AC  38_70AC  56_JPXNB  37_LAWB  ...   
           10       38_74AC  38_74AC  38_70AC  56_JPXNB  37_LAWB  ...   
           11            38_74AC  38_76AC  56_JPXNB  37_LAWB  30_292S   
           12            38_74AC  38_76AC  56_JPXNB  37_LAWB  30_292S   
           13            38_74AC  38_76AC  56_JPXNB  37_LAWB  30_292S   
           14            38_74AC  38_76AC  56_JPXNB  37_LAWB  30_292S   
           15       38_74AC  38_76AC  56_JPXNB  38_71SBAC  37_LAWB...   
           16       38_74AC  38_76AC  56_JPXNB  38_71SBAC  37_LAWB...   
           17       38_74AC  38_76AC  56_JPXNB  38_71SBAC  37_LAWB...   
           18       38_74AC  38_76AC  56_JPXNB  38_71SBAC  37_LAWB...   

                                                           path_modes  \
unique_id  pathnum                                                      
10036_3621 0        walk_access local_bus transfer local_bus trans...   
           1        walk_access local_bus transfer local_bus trans...   
           2        walk_access local_bus transfer local_bus trans...   
           3        walk_access local_bus transfer local_bus trans...   
           4        walk_access local_bus transfer local_bus trans...   
           5        walk_access local_bus transfer local_bus trans...   
           6        walk_access local_bus transfer local_bus trans...   
           7        walk_access local_bus transfer local_bus trans...   
           8        walk_access local_bus transfer local_bus trans...   
           9        walk_access local_bus transfer local_bus trans...   
           10       walk_access local_bus transfer local_bus trans...   
           11       walk_access local_bus transfer local_bus trans...   
           12       walk_access local_bus transfer local_bus trans...   
           13       walk_access local_bus transfer local_bus trans...   
           14       walk_access local_bus transfer local_bus trans...   
           15       walk_access local_bus transfer local_bus trans...   
           16       walk_access local_bus transfer local_bus trans...   
           17       walk_access local_bus transfer local_bus trans...   
           18       walk_access local_bus transfer local_bus trans...   

                                                      path_components  \
unique_id  pathnum                                                      
10036_3621 0        2059 walk_access _12574 12574 local_bus 38_74A...   
           1        2059 walk_access _12574 12574 local_bus 38_74A...   
           2        2059 walk_access _12574 12574 local_bus 38_74A...   
           3        2059 walk_access _12574 12574 local_bus 38_74A...   
           4        2059 walk_access _12574 12574 local_bus 38_74A...   
           5        2059 walk_access _12574 12574 local_bus 38_74A...   
           6        2059 walk_access _12574 12574 local_bus 38_74A...   
           7        2059 walk_access _12574 12574 local_bus 38_74A...   
           8        2059 walk_access _12574 12574 local_bus 38_74A...   
           9        2059 walk_access _12574 12574 local_bus 38_74A...   
           10       2059 walk_access _12574 12574 local_bus 38_74A...   
           11       2059 walk_access _125

In [ ]:
total_match = observed_path.apply(check_path_existence, pathset_df=new_pathset, unique_id='unique_id', lookup_field='path_components', axis=1)

In [ ]:
df = pd.DataFrame([route_match,mode_match,total_match]).T
df['id'] = df.index.get_level_values(0).values
df.columns = ['route_match','mode_match','total_match','unique_id']

In [ ]:
# Join those results back into the observed path file
df = pd.merge(observed_path, df)

In [ ]:
# Percent of the observed trips that are found in the pathset, based on matching routes, modes, and everything
df.mean()

# Get the probability of the corresponding path from the pathset

In [19]:
def get_path_attribute(row, pathset_df, unique_id, lookup_field, value_field):
    """
    For a chosen path, return parameter from path set.
    E.g., return path probability (value_field) for observed path, based on matching path description (lookup_field)
    
    Note: if multiple paths match parameters, return the max value from the matching set
    """
    
    if len(pathset_paths[(pathset_df[unique_id] == row[unique_id]) & \
                        (pathset_df[lookup_field] == row[lookup_field])]) > 0:
        try:
            return pathset_paths[(pathset_df[unique_id] == row[unique_id]) &  \
                             (pathset_df[lookup_field] == row[lookup_field])][value_field].values.max().index
        except:
            return 0
    else:
        return 0

In [20]:
# new_pathset['pathnum'] = new_pathset.index.get_level_values(1).values
# new_pathset['unique_id'] = new_pathset.index.get_level_values(0).values
# new_pathset = new_pathset.reset_index(drop=True)

In [10]:
# Join the probability to the new_pathset file
newest_pathset = pd.merge(new_pathset,pathset_paths,left_on=['unique_id','pathnum'], right_on=['unique_id','pathnum'])

KeyError: 'unique_id'

In [ ]:
route_based = observed_path.apply(get_path_attribute, pathset_df=newest_pathset, unique_id='unique_id', 
                                  lookup_field='path_routes', value_field='pf_probability', axis=1)

In [ ]:
entire_path = observed_path.apply(get_path_attribute, pathset_df=newest_pathset, unique_id='unique_id', 
                                  lookup_field='path_components', value_field='pf_probability', axis=1)

In [ ]:
a = observed_path[(observed_path['unique_id'] == '1095531_3_1')]['path_components'].values

In [ ]:
b = modeled_path[(modeled_path['unique_id'] == '1095531_3_1')]['path_components'].values[0]

In [ ]:
newest_pathset[newest_pathset['path_components'] == b].to_csv('test.csv')

In [ ]:
# newest_pathset[observed_path['path_components'] == observed_path[(observed_path['unique_id'] == '1095531_3_1')]['path_components'].values[0]]

In [ ]:
route_based[route_based.index == '1095531_3_1']

In [ ]:
pathset_paths[pathset_paths['chosen'] == 1]

In [ ]:
# Convert fields to list
df['path_routes_observed_list'] = df['path_routes_observed'].apply(lambda x: filter(None,x.split(" ")))
df['path_routes_model_list'] = df['path_routes_model'].apply(lambda x: filter(None,x.split(" ")))

In [ ]:
df['path_routes_observed_list'].isin(df['path_routes_model_list'].values())

In [ ]:
df['path_routes_model_list'].values()

In [ ]:
# Store the output in a new file


# Create "path routes"
obs['path_routes'] = obs['route_id'].apply(lambda x: x.strip())
obs_path_routes = pd.DataFrame(obs.groupby(['person_id','trip_list_id_num'])['path_routes'].apply(lambda x: "%s" % ' '.join(x).strip()))
# observed_paths.index = obs_path_routes.index
# observed_paths = pd.DataFrame(index=obs_path_routes.index)
# observed_paths['path_routes'] = obs_path_routes

In [ ]:
# produce observed_paths file



In [ ]:
# Store the output in a new file
observed_paths = pd.DataFrame()

# Create "path routes"
obs['path_routes'] = obs['route_id'].apply(lambda x: x.strip())
obs_path_routes = pd.DataFrame(obs.groupby(['person_id','trip_list_id_num'])['path_routes'].apply(lambda x: "%s" % ' '.join(x).strip()))
observed_paths.index = obs_path_routes.index
observed_paths['path_routes'] = obs_path_routes

In [ ]:
# Create "path_modes"
obs['path_modes'] = obs['mode'].apply(lambda x: x.strip())
obs_path_modes = pd.DataFrame(obs.groupby(['person_id','trip_list_id_num'])['mode'].apply(lambda x: "%s" % ' '.join(x).strip()))
observed_paths['path_modes'] = obs_path_modes

In [ ]:
# Create "path_components"
obs['path_components'] = obs['A_id_num'].astype('str')+" "+obs['mode']+" "+obs['route_id']
obs['path_components'] = obs['path_components'].apply(lambda x: x.strip())
obs_path_components = pd.DataFrame(obs.groupby(['person_id','trip_list_id_num'])['path_components'].apply(lambda x: "%s" % ' '.join(x).strip()))
observed_paths['path_components'] = obs_path_components

In [ ]:
# We need similar fields to compare against in the modeled data
